In [9]:
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>
    div.output_area{
        max-height:10000px;
        overflow:scroll;
    }
</style>"""))

import shapefile
from os.path import join
import numpy as np
import pygeohash as geohash
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from random import uniform
basedir="/Users/tgadf/Downloads"

Notebook Last Run Initiated: 2018-06-30 18:21:49.079045


In [2]:
from datetime import datetime as dt
import datetime
from datetime import timedelta, date
from calendar import monthrange
    
def getDateTime(dtval, dtformat='%Y-%m-%d'):
    #    datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')
    try:
        dtobj = dt.strptime(dtval, dtformat)
        return dtobj
    except:
        dtobj = None

    if dtobj is None:
        dtformats = ['%b %d, %Y', '%b %d %Y', '%Y-%m-%d %H:%M:%S']
        for dtformat in dtformats:
            try:
                dtobj = dt.strptime(dtval, dtformat)
                return dtobj
            except:
                dtobj = None
                
    return dtobj

def printDateTime(dtval,dtformat="%Y-%m-%d"):
    if isinstance(dtval,str):
        dtobj = getDateTime(dtval)
    else:
        dtobj = dtval
        
    if isinstance(dtobj, datetime.datetime):
        dtstr = dt.strftime(dtobj, dtformat)
    elif isinstance(dtobj, datetime.date):
        dtstr = dt.strftime(dtobj, dtformat)
    else:
        print("Can not print object {0}".format(dtval))
        dtstr = dtval
              
    return dtstr

def clock(comment=None, showTime=True):
    val = dt.now()
    if comment is None:
        comment = "Begin"
    if showTime is True:
        print("Current Time is {0} for {1}".format(printDateTime(val, "%a %b %d, %Y %H:%M:%S"), comment))
    return val


def remaining(totaltime, current, total):
    # Total Time = total/eventsPerTime
    # Remain = Total Time - time
    try:
        eventsPerTime = float(current)/totaltime
        estimate = total/eventsPerTime - totaltime
    except:
        estimate = "?"
    return estimate


def elapsed(start,comment=None,showTime=True,current=None,total=None):
    if comment is None:
        comment = "End"
    end = clock(comment=comment, showTime=showTime)
    dtime = end-start
    dtime = dtime.seconds
    if dtime>60:
        minutes = round(dtime/60.0, 1)
        if all([current,total]):
            remain  = round(remaining(minutes, current, total), 1)
        else:
            remain  = "?"
        print("Process [{0}] took {1} minutes with {2} remaining".format(comment, minutes, remain))
    else:
        seconds = round(dtime/1.0, 0)
        if all([current,total]):
            remain  = round(remaining(seconds, current, total), 1)
        else:
            remain  = "?"
        print("Process [{0}] took {1} seconds with {2} remaining.".format(comment, seconds, remain))

In [3]:
_base32 = '0123456789bcdefghjkmnpqrstuvwxyz'
_base32_map = {}
for i in range(len(_base32)):
	_base32_map[_base32[i]] = i
    
def _encode_i2c(lat,lon,lat_length,lon_length):
	precision = int((lat_length+lon_length)/5)
	if lat_length < lon_length:
		a = lon
		b = lat
	else:
		a = lat
		b = lon
	
	boost = (0,1,4,5,16,17,20,21)
	ret = ''
	for i in range(precision):
		ret+=_base32[(boost[a&7]+(boost[b&3]<<1))&0x1F]
		t = a>>3
		a = b>>2
		b = t
	
	return ret[::-1]


def _decode_c2i(hashcode):
	lon = 0
	lat = 0
	bit_length = 0
	lat_length = 0
	lon_length = 0
	for i in hashcode:
		t = _base32_map[i]
		if bit_length%2==0:
			lon = lon<<3
			lat = lat<<2
			lon += (t>>2)&4
			lat += (t>>2)&2
			lon += (t>>1)&2
			lat += (t>>1)&1
			lon += t&1
			lon_length+=3
			lat_length+=2
		else:
			lon = lon<<2
			lat = lat<<3
			lat += (t>>2)&4
			lon += (t>>2)&2
			lat += (t>>1)&2
			lon += (t>>1)&1
			lat += t&1
			lon_length+=2
			lat_length+=3
		
		bit_length+=5
	
	return (lat,lon,lat_length,lon_length)

def neighbors(hashcode):
	#if _geohash and len(hashcode)<25:
	#	return _geohash.neighbors(hashcode)
	
	(lat,lon,lat_length,lon_length) = _decode_c2i(hashcode)
	ret = []
	tlat = lat
	for tlon in (lon-1, lon+1):
		code = _encode_i2c(tlat,tlon,lat_length,lon_length)
		if code:
			ret.append(code)
	
	tlat = lat+1
	if not tlat >> lat_length:
		for tlon in (lon-1, lon, lon+1):
			ret.append(_encode_i2c(tlat,tlon,lat_length,lon_length))
	
	tlat = lat-1
	if tlat >= 0:
		for tlon in (lon-1, lon, lon+1):
			ret.append(_encode_i2c(tlat,tlon,lat_length,lon_length))
	
	return ret

In [4]:
gX = 'bcdefghjkmnpqrstuvwxyz0123456789'

geoID = 4
latID = 13
lngID = 14
wtrID = 11
lndID = 12
ids   = [geoID, latID, lngID, lndID, wtrID]

def splitGeoID(geoID):
    stateID  = geoID[:2]
    shape1ID = geoID[2:5]
    shape2ID = geoID[5:11]
    shape3ID = geoID[11:15]
    return [stateID, shape1ID, shape2ID, shape3ID]

def getRecordInfo(record):
    info = [record[x] for x in ids]
    info[1] = float(info[1])
    info[2] = float(info[2])
    return info

def getBBox(shape):
    bbox = [float('%.6f' % coord) for coord in shape.bbox]
    return bbox

In [ ]:
from glob import glob
from pickle import load, dump
from os.path import splitext
files=glob("tl_2017_*_tabblock10.p")
for ifile in files:
    print("Loading {0}".format(ifile))
    shapeData = load(open(ifile, "rb"))
    stdir = splitext(ifile)[0]
    geoData = {}
    shapeFile = join(basedir, stdir, stdir)
    print("Loading {0}".format(shapeFile))
    sf = shapefile.Reader(shapeFile)

    shapes = sf.shapes()
    Nshapes = len(shapes)
    
    for shapeRec in sf.iterShapeRecords():
        record = shapeRec.record
        info   = getRecordInfo(record)
        geoID  = info[0]
        
        shape  = shapeRec.shape
        poly   = Polygon(shape.points)
        geos   = shapeData[geoID]["Hashs"]
        
        geoData[geoID] = set()
        for geo in geos:
            for letter in gX:
                geo7 = "".join([geo,letter])
                x,y  = geohash.decode_exactly(geo7)[:2]
                pnt  = Point(x,y)
                if poly.contains(pnt):
                    geoData[geoID].add(geo7)
                    
        Nsd = len(geoData)
        if Nsd % 10000 == 0 or Nsd == 250 or Nsd == 500 or Nsd == 1000 or Nsd == 5000 or Nsd == Nshapes - 500:
            elapsed(start, "Analyzed {0}/{1} shapes".format(Nsd, Nshapes), showTime=False, current=Nsd, total=Nshapes)
            #print("  Shape ({0}) with ID {1} contains {2} geohashs ({3} MC) and area {4} radians.".format(len(shapeData), geoID, len(geoHashs), cnt, round(area,6)))

    outfile="{0}-geo.p".format(stdir)
    elapsed(start, "Analyzed {0} shapes".format(Nshapes), showTime=True, current=Nshapes, total=Nshapes)
    dump(geoData, open(outfile, "wb"))
    print("Saved shape data to {0}.p".format(stdir))
    print("\n\n\n")

Loading tl_2017_01_tabblock10.p
Loading /Users/tgadf/Downloads/shapefiles/tl_2017_01_tabblock10/tl_2017_01_tabblock10
Process [Analyzed 250/252266 shapes] took 49.6 minutes with 50000.0 remaining
Process [Analyzed 500/252266 shapes] took 49.7 minutes with 25025.5 remaining
Process [Analyzed 1000/252266 shapes] took 49.9 minutes with 12538.2 remaining
Process [Analyzed 5000/252266 shapes] took 51.5 minutes with 2546.8 remaining
Process [Analyzed 10000/252266 shapes] took 53.1 minutes with 1286.4 remaining
Process [Analyzed 20000/252266 shapes] took 56.3 minutes with 653.8 remaining
Process [Analyzed 30000/252266 shapes] took 59.7 minutes with 442.3 remaining
Process [Analyzed 40000/252266 shapes] took 61.7 minutes with 327.4 remaining
Process [Analyzed 50000/252266 shapes] took 62.5 minutes with 252.8 remaining
Process [Analyzed 60000/252266 shapes] took 63.2 minutes with 202.5 remaining
Process [Analyzed 70000/252266 shapes] took 63.9 minutes with 166.4 remaining
Process [Analyzed 8000

Saved shape data to tl_2017_05_tabblock10.p




Loading tl_2017_06_tabblock10.p
Loading /Users/tgadf/Downloads/shapefiles/tl_2017_06_tabblock10/tl_2017_06_tabblock10
Process [Analyzed 250/710145 shapes] took 90.0 minutes with 255562.2 remaining
Process [Analyzed 500/710145 shapes] took 90.0 minutes with 127736.1 remaining
Process [Analyzed 1000/710145 shapes] took 90.0 minutes with 63823.1 remaining
Process [Analyzed 5000/710145 shapes] took 90.2 minutes with 12720.8 remaining
Process [Analyzed 10000/710145 shapes] took 90.3 minutes with 6322.3 remaining
Process [Analyzed 20000/710145 shapes] took 90.7 minutes with 3129.8 remaining
Process [Analyzed 30000/710145 shapes] took 90.9 minutes with 2060.8 remaining
Process [Analyzed 40000/710145 shapes] took 91.1 minutes with 1526.3 remaining
Process [Analyzed 50000/710145 shapes] took 91.3 minutes with 1205.4 remaining
Process [Analyzed 60000/710145 shapes] took 91.5 minutes with 991.5 remaining
Process [Analyzed 70000/710145 shapes] took 9

Process [Analyzed 200000/201062 shapes] took 109.4 minutes with 0.6 remaining
Process [Analyzed 200562/201062 shapes] took 109.4 minutes with 0.3 remaining
Current Time is Sun Jun 24, 2018 21:52:01 for Analyzed 201062 shapes
Process [Analyzed 201062 shapes] took 109.4 minutes with 0.0 remaining
Saved shape data to tl_2017_08_tabblock10.p




Loading tl_2017_09_tabblock10.p
Loading /Users/tgadf/Downloads/shapefiles/tl_2017_09_tabblock10/tl_2017_09_tabblock10
Process [Analyzed 250/67578 shapes] took 109.5 minutes with 29489.7 remaining
Process [Analyzed 500/67578 shapes] took 109.5 minutes with 14690.1 remaining
Process [Analyzed 1000/67578 shapes] took 109.6 minutes with 7296.9 remaining
Process [Analyzed 5000/67578 shapes] took 110.1 minutes with 1378.0 remaining
Process [Analyzed 10000/67578 shapes] took 110.3 minutes with 635.1 remaining
Process [Analyzed 20000/67578 shapes] took 110.8 minutes with 263.6 remaining
Process [Analyzed 30000/67578 shapes] took 111.3 minutes with 139.4 re

Process [Analyzed 40000/291086 shapes] took 209.9 minutes with 1317.6 remaining
Process [Analyzed 50000/291086 shapes] took 212.8 minutes with 1026.1 remaining
Process [Analyzed 60000/291086 shapes] took 214.2 minutes with 825.0 remaining
Process [Analyzed 70000/291086 shapes] took 214.8 minutes with 678.4 remaining
Process [Analyzed 80000/291086 shapes] took 215.6 minutes with 568.9 remaining
Process [Analyzed 90000/291086 shapes] took 216.3 minutes with 483.3 remaining
Process [Analyzed 100000/291086 shapes] took 217.0 minutes with 414.7 remaining
Process [Analyzed 110000/291086 shapes] took 217.7 minutes with 358.4 remaining
Process [Analyzed 120000/291086 shapes] took 218.3 minutes with 311.2 remaining
Process [Analyzed 130000/291086 shapes] took 219.2 minutes with 271.6 remaining
Process [Analyzed 140000/291086 shapes] took 219.9 minutes with 237.3 remaining
Process [Analyzed 150000/291086 shapes] took 220.5 minutes with 207.4 remaining
Process [Analyzed 160000/291086 shapes] took

Process [Analyzed 360000/451554 shapes] took 254.9 minutes with 64.8 remaining
Process [Analyzed 370000/451554 shapes] took 255.4 minutes with 56.3 remaining
Process [Analyzed 380000/451554 shapes] took 255.9 minutes with 48.2 remaining
Process [Analyzed 390000/451554 shapes] took 256.3 minutes with 40.5 remaining
Process [Analyzed 400000/451554 shapes] took 256.8 minutes with 33.1 remaining
Process [Analyzed 410000/451554 shapes] took 257.3 minutes with 26.1 remaining
Process [Analyzed 420000/451554 shapes] took 257.7 minutes with 19.4 remaining
Process [Analyzed 430000/451554 shapes] took 258.2 minutes with 12.9 remaining
Process [Analyzed 440000/451554 shapes] took 258.7 minutes with 6.8 remaining
Process [Analyzed 450000/451554 shapes] took 259.2 minutes with 0.9 remaining
Process [Analyzed 451054/451554 shapes] took 259.2 minutes with 0.3 remaining
Current Time is Mon Jun 25, 2018 00:21:50 for Analyzed 451554 shapes
Process [Analyzed 451554 shapes] took 259.2 minutes with 0.0 rema

Process [Analyzed 210000/238600 shapes] took 288.2 minutes with 39.3 remaining
Process [Analyzed 220000/238600 shapes] took 288.5 minutes with 24.4 remaining
Process [Analyzed 230000/238600 shapes] took 288.7 minutes with 10.8 remaining
Process [Analyzed 238100/238600 shapes] took 288.9 minutes with 0.6 remaining
Current Time is Mon Jun 25, 2018 00:51:27 for Analyzed 238600 shapes
Process [Analyzed 238600 shapes] took 288.9 minutes with 0.0 remaining
Saved shape data to tl_2017_20_tabblock10.p




Loading tl_2017_21_tabblock10.p
Loading /Users/tgadf/Downloads/shapefiles/tl_2017_21_tabblock10/tl_2017_21_tabblock10
Process [Analyzed 250/161672 shapes] took 289.2 minutes with 186733.0 remaining
Process [Analyzed 500/161672 shapes] took 289.2 minutes with 93221.9 remaining
Process [Analyzed 1000/161672 shapes] took 289.4 minutes with 46498.5 remaining
Process [Analyzed 5000/161672 shapes] took 290.9 minutes with 9115.2 remaining
Process [Analyzed 10000/161672 shapes] took 292.8 minutes wit

In [7]:
pickle.dump()

TypeError: Required argument 'obj' (pos 1) not found

In [ ]:
gX = 'bcdefghjkmnpqrstuvwxyz0123456789'

In [15]:
geohash.eastern('fdbe6x')

'xebm5kup6c5p'

In [5]:
data = {}
from glob import glob
from pickle import load, dump
from os.path import splitext
files=glob("tl_2017_*_tabblock10.p")
for ifile in files:
    print("Loading {0}".format(ifile))
    shapeData = load(open(ifile, "rb"))
    stdir = splitext(ifile)[0]
    geo7file = stdir+"-geo.p"
    print("Loading {0}".format(geo7file))
    geo7Data = load(open(geo7file, "rb"))
        
    for geoID,geo7s in geo7Data.items():
        areadata = shapeData[geoID]
        data[geoID] = areadata
        data[geoID]["Hashs"] = geo7s
    print(  "--> {0}".format(len(data)))

Loading tl_2017_01_tabblock10.p
Loading tl_2017_01_tabblock10-geo.p
--> 252266
Loading tl_2017_02_tabblock10.p
Loading tl_2017_02_tabblock10-geo.p
--> 297558
Loading tl_2017_04_tabblock10.p
Loading tl_2017_04_tabblock10-geo.p
--> 539224
Loading tl_2017_05_tabblock10.p
Loading tl_2017_05_tabblock10-geo.p
--> 725435
Loading tl_2017_06_tabblock10.p
Loading tl_2017_06_tabblock10-geo.p
--> 1435580
Loading tl_2017_08_tabblock10.p
Loading tl_2017_08_tabblock10-geo.p
--> 1636642
Loading tl_2017_09_tabblock10.p
Loading tl_2017_09_tabblock10-geo.p
--> 1704220
Loading tl_2017_10_tabblock10.p
Loading tl_2017_10_tabblock10-geo.p
--> 1728335
Loading tl_2017_11_tabblock10.p
Loading tl_2017_11_tabblock10-geo.p
--> 1734842
Loading tl_2017_12_tabblock10.p
Loading tl_2017_12_tabblock10-geo.p
--> 2219323
Loading tl_2017_13_tabblock10.p
Loading tl_2017_13_tabblock10-geo.p
--> 2510409
Loading tl_2017_15_tabblock10.p
Loading tl_2017_15_tabblock10-geo.p
--> 2535425
Loading tl_2017_16_tabblock10.p
Loading tl_2

In [7]:
import pickle
import gzip

def save_zipped_pickle(obj, filename, protocol=-1):
    with gzip.open(filename, 'wb') as f:
        pickle.dump(obj, f, protocol)
        
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object

In [8]:
save_zipped_pickle(data, "geoIDdata.p")

# Places

In [20]:
def getShapeGeos(poly, prec, geo, geos, ignores):
    print("getShapeGeos({0})".format(geo))
    if geo in ignores:
        return geos, ignores
    
    x,y  = geohash.decode_exactly(geo)[:2]
    pnt  = Point(x,y)
    if poly.contains(pnt) is False:
        ignores.add(geo)
        return geos, ignores
    else:
        geos.add(geo)
        print("Adding {0} to make {1} total geos".format(geo, len(geos)))
        for neighbor in geohash.neighbor(geo):
            geos, ignores = getShapeGeos(poly=poly, prec=prec, geo=neighbor, geos=geos, ignores=ignores)

In [21]:
statedata="cb_2017_01_place_500k"

In [40]:
shapeFile = join(basedir, "places", statedata)
print(shapeFile)
from os.path import isdir
from random import random
if isdir(shapeFile):
    print("Loading {0}".format(shapeFile))
    sf = shapefile.Reader(join(shapeFile, statedata))

    shapes = sf.shapes()
    Nshapes = len(shapes)
    print(Nshapes)
    fields = sf.fields
    print(fields)
    n = 0
    for shapeRec in sf.iterShapeRecords():
        n += 1
        record = shapeRec.record
        shape  = shapeRec.shape
        poly   = Polygon(shape.points)
        print(poly.area)

        center  = poly.centroid
        pnt     = center
        print(center)
        print(poly.contains(center))
        boundary = poly.bounds
        xrange   = abs(boundary[0] - boundary[2])
        yrange   = abs(boundary[1] - boundary[3])
        print(boundary)
        i = 0
        while not poly.contains(center):
            pnt = Point(center.x + xrange*(random() - 0.5), center.y + yrange*(random() - 0.5))
            print(i,pnt)
            i += 1
        center  = pnt
        print(poly.contains(center))
        1/0
        long0   = center.x
        lat0    = center.y
        geo     = geohash.encode(latitude=lat0, longitude=long0, precision=7)
        geos    = set()
        ignores = set()
        geos, ignores = getShapeGeos(poly, 7, geo, geos, ignores)
        print(len(geos), len(ignores))
        1/0
        
        geoid = record[4]
        name  = record[5]
        lsad  = record[6] # legal/statistical area description
        print(record)
        if n > 100:
            break


/Users/tgadf/Downloads/places/cb_2017_01_place_500k
Loading /Users/tgadf/Downloads/places/cb_2017_01_place_500k
585
[('DeletionFlag', 'C', 1, 0), ['STATEFP', 'C', 2, 0], ['PLACEFP', 'C', 5, 0], ['PLACENS', 'C', 8, 0], ['AFFGEOID', 'C', 16, 0], ['GEOID', 'C', 7, 0], ['NAME', 'C', 100, 0], ['LSAD', 'C', 2, 0], ['ALAND', 'N', 14, 0], ['AWATER', 'N', 14, 0]]
0.005993789882999948
POINT (-86.45533525622612 32.46352882667938)
False
(-86.55608, 32.406461, -86.36448999999999, 32.511727)
0 POINT (-86.50802234564073 32.51391450762301)
1 POINT (-86.42235581012841 32.47538686515909)
2 POINT (-86.50342193480432 32.46927487358481)
3 POINT (-86.51211027541557 32.49227735898367)
4 POINT (-86.47869405697138 32.48329132121468)
5 POINT (-86.41088620931163 32.47826338016723)
6 POINT (-86.50826188028338 32.51441176997074)
7 POINT (-86.46731300700797 32.51039500236324)
8 POINT (-86.44058696183615 32.46631852564393)
9 POINT (-86.51954629518929 32.48089614542041)
10 POINT (-86.48667291831697 32.5135684935679)


184 POINT (-86.50683567858349 32.5064146544721)
185 POINT (-86.37592852821578 32.51402362534242)
186 POINT (-86.37363396686848 32.45546032911943)
187 POINT (-86.55007142928046 32.48887338533885)
188 POINT (-86.44913797090669 32.42819258437304)
189 POINT (-86.45467607828505 32.46356975038754)
190 POINT (-86.48861597482505 32.4226970659622)
191 POINT (-86.50418958012675 32.50373854818586)
192 POINT (-86.43544669181243 32.46316366570811)
193 POINT (-86.37083821839306 32.45891318362267)
194 POINT (-86.55030524307429 32.45604453744603)
195 POINT (-86.55018482893206 32.47615558007492)
196 POINT (-86.50252805998474 32.45463558764643)
197 POINT (-86.42984430726514 32.43264761691271)
198 POINT (-86.39899942028525 32.476273390661)
199 POINT (-86.41784811293707 32.47120500164083)
200 POINT (-86.45485512781218 32.44834048472959)
201 POINT (-86.51269930417516 32.49768931956669)
202 POINT (-86.54637477372717 32.41757739341409)
203 POINT (-86.36084055971583 32.43636149480289)
204 POINT (-86.437650222

424 POINT (-86.44416252397711 32.49754973562939)
425 POINT (-86.54540734743615 32.50287086365227)
426 POINT (-86.51365252788736 32.48339497472386)
427 POINT (-86.46384182820051 32.47171634417176)
428 POINT (-86.36312435468163 32.46697235159174)
429 POINT (-86.54458683485605 32.43286829210523)
430 POINT (-86.39385089098238 32.46974670010895)
431 POINT (-86.47560396736134 32.43656576820699)
432 POINT (-86.50855153202104 32.45600185518486)
433 POINT (-86.46743221255993 32.45536518570519)
434 POINT (-86.45744052699591 32.47114283627596)
435 POINT (-86.48161977442516 32.50824380496777)
436 POINT (-86.52294671995662 32.45380292601888)
437 POINT (-86.51646144644326 32.48506650341362)
438 POINT (-86.50271654157751 32.41485356338789)
439 POINT (-86.45100049604272 32.48569257625602)
440 POINT (-86.37159192118622 32.47936770444585)
441 POINT (-86.42638392079823 32.50974559136418)
442 POINT (-86.44823502470159 32.44558045490592)
443 POINT (-86.52556048255107 32.46645076531603)
444 POINT (-86.38127

659 POINT (-86.44079536560325 32.50695854582252)
660 POINT (-86.52385076250629 32.45133732480551)
661 POINT (-86.36779016736409 32.45045773059589)
662 POINT (-86.48426562949035 32.48339220791471)
663 POINT (-86.37825947181534 32.48536202727935)
664 POINT (-86.37338666929705 32.42642289639171)
665 POINT (-86.49119551782293 32.4703985074725)
666 POINT (-86.45260680779127 32.48421365094158)
667 POINT (-86.42069329350345 32.45766419621268)
668 POINT (-86.36700524967017 32.50030040512089)
669 POINT (-86.39874462616889 32.47398769089013)
670 POINT (-86.53984046234983 32.42322137432919)
671 POINT (-86.48399730664008 32.5054199120665)
672 POINT (-86.37081621260997 32.4919879139455)
673 POINT (-86.54003962711106 32.4611991386283)
674 POINT (-86.43034227308476 32.50445011183672)
675 POINT (-86.51565617006676 32.48056282244609)
676 POINT (-86.52003524986057 32.51217293252834)
677 POINT (-86.53664231210566 32.43470724372725)
678 POINT (-86.53364255677552 32.48104401089283)
679 POINT (-86.382877949

897 POINT (-86.44673969448225 32.48215330471005)
898 POINT (-86.42727933457938 32.47075464571886)
899 POINT (-86.51180023048968 32.41844330196986)
900 POINT (-86.44252284820743 32.41547103053566)
901 POINT (-86.44411918761752 32.4786455857034)
902 POINT (-86.49675951727072 32.51508026919222)
903 POINT (-86.52701436362065 32.47948432823681)
904 POINT (-86.40296289302448 32.47905216986122)
905 POINT (-86.53354413495387 32.44458414694243)
906 POINT (-86.47856175449058 32.49931319995182)
907 POINT (-86.40900603933815 32.51100294059161)
908 POINT (-86.38872745599363 32.42710684622426)
909 POINT (-86.5194573635134 32.44155287260073)
910 POINT (-86.43353797341371 32.43136557584912)
911 POINT (-86.51118997303823 32.42102752397552)
912 POINT (-86.46474240169648 32.45453248530632)
913 POINT (-86.41433594901451 32.50805873124775)
914 POINT (-86.44482843971697 32.42053116314136)
915 POINT (-86.43201484595096 32.42553618063651)
916 POINT (-86.38986929657239 32.46145190971262)
917 POINT (-86.4359918

...

17493 POINT (-86.4548285874359 32.49759539000411)
17494 POINT (-86.50683552534694 32.49022671172473)
17495 POINT (-86.42207569496267 32.48589117282399)
17496 POINT (-86.41873158559426 32.48800195663444)
17497 POINT (-86.53494714045067 32.47850090771939)
17498 POINT (-86.46858833723613 32.49338449409949)
17499 POINT (-86.47702555133321 32.50017319269128)
17500 POINT (-86.36825572607725 32.46138800182604)
17501 POINT (-86.49533297454261 32.48584270931453)
17502 POINT (-86.54985710815274 32.41847470894334)
17503 POINT (-86.54107681734128 32.47130703613885)
17504 POINT (-86.43556852749786 32.48784970565686)
17505 POINT (-86.4805429213969 32.50217503517977)
17506 POINT (-86.43303285133118 32.49456526170182)
17507 POINT (-86.42776480210331 32.47746736469592)
17508 POINT (-86.36178398338193 32.50743728293006)
17509 POINT (-86.54840109611403 32.48328909548064)
17510 POINT (-86.41203176985678 32.48618798694669)
17511 POINT (-86.48870122657254 32.48858829819704)
17512 POINT (-86.46837645150823 3

17730 POINT (-86.49784099910651 32.44917436599506)
17731 POINT (-86.43354725244539 32.46242247556784)
17732 POINT (-86.37121381795959 32.48557299161769)
17733 POINT (-86.39557172370183 32.50114844968812)
17734 POINT (-86.37967211376197 32.42523623918601)
17735 POINT (-86.36290715825315 32.41445669747429)
17736 POINT (-86.54744455425495 32.41476008306521)
17737 POINT (-86.48043429294026 32.43465305226989)
17738 POINT (-86.43812488866791 32.4720827262315)
17739 POINT (-86.44162754129805 32.47073763122833)
17740 POINT (-86.36211714877109 32.47639213595642)
17741 POINT (-86.5025740354226 32.43704083349206)
17742 POINT (-86.42381832521625 32.42591820208264)
17743 POINT (-86.53218830577993 32.49061352056936)
17744 POINT (-86.4455831843204 32.496603887873)
17745 POINT (-86.49551551274116 32.48374481533455)
17746 POINT (-86.4987059652931 32.47986524812808)
17747 POINT (-86.36129996613197 32.46836196678154)
17748 POINT (-86.48899728590773 32.4753273812544)
17749 POINT (-86.44299863587025 32.495

17948 POINT (-86.36626881732252 32.44268562472442)
17949 POINT (-86.44110989030536 32.472759812549)
17950 POINT (-86.54835683198854 32.43758836374398)
17951 POINT (-86.477343778438 32.41541753681731)
17952 POINT (-86.37991109242023 32.51472414543879)
17953 POINT (-86.53578039450404 32.50660034826915)
17954 POINT (-86.45236983599342 32.49424726320805)
17955 POINT (-86.41043520417125 32.49421257384591)
17956 POINT (-86.49568022089925 32.49305930454659)
17957 POINT (-86.41655296012868 32.4397388832372)
17958 POINT (-86.36486167312906 32.47659218768428)
17959 POINT (-86.46577258745991 32.43213839848728)
17960 POINT (-86.42872504708319 32.45099668124755)
17961 POINT (-86.41959079547151 32.43944277742264)
17962 POINT (-86.49664888862513 32.51290485376994)
17963 POINT (-86.46781019043711 32.45122636336325)
17964 POINT (-86.4347779523479 32.48897398122384)
17965 POINT (-86.43021654381333 32.50748303394943)
17966 POINT (-86.48586215877047 32.43882559335627)
17967 POINT (-86.41403740474003 32.47

18187 POINT (-86.46936271760282 32.49575817656741)
18188 POINT (-86.37846451235937 32.46420853590458)
18189 POINT (-86.4881905774253 32.4316134145057)
18190 POINT (-86.52359499121597 32.41364877184854)
18191 POINT (-86.38726246715608 32.43855928235897)
18192 POINT (-86.42179705430821 32.48907708448889)
18193 POINT (-86.54304635253368 32.45227396884113)
18194 POINT (-86.48011398121979 32.50664476502556)
18195 POINT (-86.41756098612751 32.44216985338165)
18196 POINT (-86.49333868156664 32.41309463101524)
18197 POINT (-86.42027530255882 32.46251194955322)
18198 POINT (-86.36896857114378 32.51426254072791)
18199 POINT (-86.54738300595889 32.49185254482297)
18200 POINT (-86.43706929663604 32.51133757065634)
18201 POINT (-86.45421158182798 32.43710820684888)
18202 POINT (-86.4307132626061 32.50052516594295)
18203 POINT (-86.43471406431343 32.45804980711496)
18204 POINT (-86.39587679113372 32.47737483232055)
18205 POINT (-86.47092404777628 32.43906391819039)
18206 POINT (-86.41663034282051 32

KeyboardInterrupt: 